<a href="https://colab.research.google.com/github/andreaaraldo/machine-learning-for-networks/blob/master/08.predictive-maintenance/Predictive-maintenance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Need to configure packages

We first need to revert to an older version of scikit-learn, for a compatibility issue with `scikit-survival`, the library we are going to use for our Survival Analysis

In [1]:
!pip uninstall scikit-learn -y
!pip install scikit-learn==0.22
!pip install scikit-survival

Uninstalling scikit-learn-0.22:
  Successfully uninstalled scikit-learn-0.22
  Using cached https://files.pythonhosted.org/packages/2e/d0/860c4f6a7027e00acff373d9f5327f4ae3ed5872234b3cbdd7bcb52e5eff/scikit_learn-0.22-cp36-cp36m-manylinux1_x86_64.whl


In [0]:
import pandas as pd

from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

# Load the dataset

In [3]:
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
! wget https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/training_set.csv

--2020-05-19 23:11:01--  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4437889 (4.2M) [text/plain]
Saving to: ‘test_set.csv’

test_set.csv          0%[                    ]  12.05K  --.-KB/s    in 0.001s  

2020-05-19 23:11:02 (16.9 MB/s) - Read error at byte 12339/4437889 (Connection reset by peer). Retrying.

--2020-05-19 23:11:03--  (try: 2)  https://raw.githubusercontent.com/andreaaraldo/machine-learning-for-networks/master/08.predictive-maintenance/dataset/transformed/test_set.csv
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response

In [0]:
df_trn = pd.read_csv("training_set.csv")
df_test = pd.read_csv("training_set.csv")

In [9]:
df_trn.head()

,setting1,setting2,setting3,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14,s15,s16,s17,s18,s19,s20,s21,remaining_duration,failure_observed
0,0.459770,0.166667,0.0,0.0,0.183735,0.406802,0.309757,0.0,1.0,0.726248,0.242424,0.109755,0.0,0.369048,0.633262,0.205882,0.199608,0.363986,0.0,0.333333,0.0,0.0,0.713178,0.724662,191,True
1,0.609195,0.250000,0.0,0.0,0.283133,0.453019,0.352633,0.0,1.0,0.628019,0.212121,0.100242,0.0,0.380952,0.765458,0.279412,0.162813,0.411312,0.0,0.333333,0.0,0.0,0.666667,0.731014,190,True
2,0.252874,0.750000,0.0,0.0,0.343373,0.369523,0.370527,0.0,1.0,0.710145,0.272727,0.140043,0.0,0.250000,0.795309,0.220588,0.171793,0.357445,0.0,0.166667,0.0,0.0,0.627907,0.621375,189,True
3,0.540230,0.500000,0.0,0.0,0.343373,0.256159,0.331195,0.0,1.0,0.740741,0.318182,0.124518,0.0,0.166667,0.889126,0.294118,0.174889,0.166603,0.0,0.333333,0.0,0.0,0.573643,0.662386,188,True
4,0.390805,0.333333,0.0,0.0,0.349398,0.257467,0.404625,0.0,1.0,0.668277,0.242424,0.149960,0.0,0.255952,0.746269,0.235294,0.174734,0.402078,0.0,0.416667,0.0,0.0,0.589147,0.704502,187,True


In [10]:
X_trn = df_trn.drop(columns=["remaining_duration", "failure_observed"])
y_trn = df_trn[["failure_observed", "remaining_duration"]]

y_trn

,failure_observed,remaining_duration
0,True,191
1,True,190
2,True,189
3,True,188
4,True,187
...,...,...
20626,True,4
20627,True,3
20628,False,2
20629,True,1


In [11]:
print(f'Number of samples: {len(y_trn)}')
print(f'Number of right censored samples: {len(y_trn.query("failure_observed == False"))}')
print(f'Percentage of right censored samples: {100*len(y_trn.query("failure_observed == False"))/len(y_trn):.1f}%')

Number of samples: 20631
Number of right censored samples: 2155
Percentage of right censored samples: 10.4%


The dataset is already scaled and all the columns are numerical. We do not need to do pre-processing for that.

In [0]:
y_trn_record = y_trn.to_records(index=False)

In [17]:

model = CoxPHSurvivalAnalysis()
model.fit(X_trn, y_trn_record)

LinAlgError: ignored